## Brute force

In [ ]:
matrix = []
with open("tsp/tsp2_1248.txt", "r") as f:
    for line in f:
        row = list(map(int, line.strip().split()))
        matrix.append(row)

print(matrix)


[[0, 64, 378, 519, 434, 200], [64, 0, 318, 455, 375, 164], [378, 318, 0, 170, 265, 344], [519, 455, 170, 0, 223, 428], [434, 375, 265, 223, 0, 273], [200, 164, 344, 428, 273, 0]]


In [ ]:

size = len(matrix[0])
nodes = []

for i in range(size-1):
  nodes.append(i+1)

print(nodes)

[1, 2, 3, 4, 5]


In [ ]:
import itertools
import sys

minPath = 1000000
cost = 0
current = 0

for perm in itertools.permutations(nodes):
  for i in perm:
    cost += matrix[current][i]
    current = i
  cost += matrix[perm[-1]][0]
  minPath = min(minPath,cost)
  cost = 0
  current = 0

print(minPath)

1248


##  Código aproximativo verdadeiro

In [ ]:
# Downloading files

!pip install gdown
import gdown
import zipfile
import os

file_id = '10ZKk0earBCo1QZtc-PBvaWUt2Pe8rE7k'
output = 'tsp.zip'

gdown.download(f'https://drive.google.com/uc?id={file_id}', output, quiet=False)

with zipfile.ZipFile(output, 'r') as zip_ref:
    zip_ref.extractall('./')

Downloading...
From: https://drive.google.com/uc?id=10ZKk0earBCo1QZtc-PBvaWUt2Pe8rE7k
To: /content/tsp.zip
100%|██████████| 5.22k/5.22k [00:00<00:00, 9.91MB/s]


In [ ]:
# reading file and transforming matrix into np.array
import numpy as np
import sys

temp_matrix = []
with open("./tsp/tsp4_7013.txt", "r") as f:
    for line in f:
        row = list(map(int, line.strip().split()))
        temp_matrix.append(row)

for i in range(len(temp_matrix)): # If nodes have no connection, set 0 to "infinite"
    for j in range(len(temp_matrix[i])):
        if i != j and temp_matrix[i][j] == 0:
            temp_matrix[i][j] = int(1e9)


matrix = np.array(temp_matrix)
np.fill_diagonal(matrix, 0)   # Set diagonal back to 0
print(matrix.shape)

(44, 44)


In [ ]:
np.set_printoptions(threshold=np.inf)
print(matrix)

[[         0        509        501        312       1019        736
         656         60       1039        726       2314        479
         448        479        619        150        342        323
         635        604        596        202 1000000000        509
         501        312       1019        736        656         60
        1039        726       2314        479        448        479
         619        150        342        323        635        604
         596        202]
 [       509          0        126        474       1526       1226
        1133        532       1449       1122       2789        958
         941        978       1127        542        246        510
        1047       1021       1010        364        509 1000000000
         126        474       1526       1226       1133        532
        1449       1122       2789        958        941        978
        1127        542        246        510       1047       1021
        1010        364

In [ ]:
# Used to connect graphs that are not full connected (tsp4_7013)
def floyd_warshall(matrix):
  n = len(matrix)
  complete_graph = np.copy(matrix)

  for k in range(n):
    for i in range(n):
      for j in range(n):
        complete_graph[i][j] = min(complete_graph[i][j], complete_graph[i][k] + complete_graph[k][j])

  return complete_graph

In [ ]:
matrix = floyd_warshall(matrix)

In [ ]:
# Prim's algorithm to create MST

from ast import And
import heapq

def prim(matrix):
  pqueue = []
  n = len(matrix) # number of nodes
  m = n - 1 # number of edges
  edge_count = 0
  mst = []
  visited = [False] * n

  visited[0] = True

  for i in range(n):
    if matrix[0][i] != 0:
      heapq.heappush(pqueue, (matrix[0][i],0,i))


  while pqueue and edge_count != m:
    e = heapq.heappop(pqueue)

    if visited[e[2]]:
      continue

    mst.append(e)
    edge_count += 1



    visited[e[2]] = True
    for i in range(n):
      if matrix[e[2]][i] != 0 and not visited[i]:
        heapq.heappush(pqueue, (matrix[e[2]][i], e[2], i))


  if edge_count != m:
    return []

  return mst

In [ ]:
# Printing MST

mst = prim(matrix)
for weight, u, v in mst:
     print(f"{u} - {v}, weight = {weight}")

0 - 7, weight = 60
0 - 29, weight = 60
7 - 22, weight = 60
0 - 15, weight = 150
0 - 37, weight = 150
7 - 21, weight = 194
21 - 43, weight = 1
21 - 16, weight = 148
21 - 38, weight = 148
21 - 3, weight = 171
3 - 17, weight = 37
3 - 39, weight = 37
17 - 25, weight = 37
16 - 1, weight = 246
1 - 2, weight = 126
1 - 24, weight = 126
2 - 23, weight = 126
15 - 12, weight = 406
12 - 13, weight = 52
12 - 35, weight = 52
13 - 34, weight = 52
12 - 11, weight = 68
12 - 33, weight = 68
11 - 6, weight = 177
6 - 5, weight = 115
5 - 28, weight = 115
6 - 27, weight = 115
11 - 19, weight = 214
19 - 20, weight = 14
19 - 42, weight = 14
20 - 41, weight = 14
19 - 18, weight = 33
19 - 40, weight = 33
18 - 9, weight = 96
18 - 31, weight = 96
13 - 14, weight = 237
13 - 36, weight = 237
9 - 8, weight = 328
9 - 30, weight = 328
14 - 4, weight = 401
14 - 26, weight = 401
8 - 10, weight = 1387
8 - 32, weight = 1387


In [ ]:
# transform MST into a matrix (we need to traverse it)

mst_matrix = np.full((len(matrix), len(matrix)), 0)

for w, u, v in mst:
  mst_matrix[u][v] = w
  mst_matrix[v][u] = w

print (mst_matrix)

[[   0    0    0    0    0    0    0   60    0    0    0    0    0    0
     0  150    0    0    0    0    0    0    0    0    0    0    0    0
     0   60    0    0    0    0    0    0    0  150    0    0    0    0
     0    0]
 [   0    0  126    0    0    0    0    0    0    0    0    0    0    0
     0    0  246    0    0    0    0    0    0    0  126    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0]
 [   0  126    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0  126    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0   37    0    0    0  171    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0   37    0    0
     0    0]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
   401    0 

In [ ]:
# Traversing MST

def pre_order_traversal (matrix, current, parent, pre_order_array, visited):
  visited[current] = True

  pre_order_array.append(current)

  for v in range(len(matrix)):
    if matrix[current][v] != 0 and not visited[v]:
      pre_order_traversal(matrix, v, current, pre_order_array, visited)




In [ ]:
n = len(mst_matrix)
current = 0
parent = 0
pre_order_array = []
visited = [False] * n

pre_order_traversal(mst_matrix, current, parent, pre_order_array, visited)

print(pre_order_array)

[0, 7, 21, 3, 17, 25, 39, 16, 1, 2, 23, 24, 38, 43, 22, 15, 12, 11, 6, 5, 28, 27, 19, 18, 9, 8, 10, 32, 30, 31, 20, 41, 40, 42, 13, 14, 4, 26, 34, 36, 33, 35, 29, 37]


In [ ]:
# Cost of final path
cost = 0

for i in range (len(pre_order_array)-1):
  print(cost)
  cost += matrix[pre_order_array[i]][pre_order_array[i+1]]
cost += matrix[pre_order_array[-1]][pre_order_array[0]]
print (cost)

0
60
254
425
462
499
536
801
1047
1173
1299
1425
1746
1894
2096
2246
2652
2720
2897
3012
3127
3242
3578
3611
3707
4035
5422
1000005422
1000006809
1000007137
1000007270
1000007284
1000007317
1000007356
1000007674
1000007911
1000008312
2000008312
2000008963
2000009250
2000009586
2000009691
2000010170
2000010526


10 -> 32 e 4 -> 26